# AWBS Project 1: Exploratory Data Analysis of Internet Traffic Performance

## Authors:
- Jakub Piotrowski 266502
- Jakub Włodarski

---

## 📘 Project Overview

This project aims to analyze and compare internet traffic performance using datasets collected in January 2021 and January 2023 by the **Federal Communications Commission (FCC)**. The datasets contain measurements like **download throughput**, **upload throughput**, and **latency**, crucial for evaluating broadband performance.

### Objectives:
1. Perform comprehensive Exploratory Data Analysis (EDA) using the CRISP-DM methodology.
2. Analyze internet traffic performance by focusing on:
   - Download and upload speeds
   - Latency
   - Relationships between metrics
3. Compare trends between 2021 and 2023.
4. Predict future internet performance using machine learning models.

### Methodology: CRISP-DM
The CRISP-DM (Cross Industry Standard Process for Data Mining) methodology guides our workflow:
- Business Understanding
- Data Understanding
- Data Preparation
- Modeling
- Evaluation
- Deployment

---

## 📂 Dataset Sources
- [2021 Dataset (FCC)](https://data.fcc.gov/download/measuring-broadband-america/2021/data-raw-2021-jan.tar.gz)
- [2023 Dataset (FCC)](https://data.fcc.gov/download/measuring-broadband-america/2023/data-raw-2023-jan.tar.gz)
- [CRISP-DM](https://www.ibm.com/docs/pl/spss-modeler/saas?topic=dm-crisp-help-overview)


## 📊 Step 1: Business Understanding

We aim to analyze and compare internet performance data from the FCC for January 2021 and 2023. Key performance indicators include download speed, upload speed, and latency.

### Objectives:
- Explore trends and performance changes over time.
- Find relationships between key metrics.
- Build predictive models for download/upload performance.

In [6]:
import os

data_2021_path = "data/data-raw-2021-jan"
data_2023_path = "data/data-raw-2023-jan"

files_2021 = os.listdir(data_2021_path)
files_2023 = os.listdir(data_2023_path)

print("2021 Files:", files_2021)
print("2023 Files:", files_2023)


2021 Files: ['202101']
2023 Files: ['202301']


In [7]:
subfolder_2023 = os.path.join(data_2023_path, '202301')

files_2023_deep = os.listdir(subfolder_2023)
for i, f in enumerate(files_2023_deep, start=1):
    print(f"{i}. {f}")


1. curr_datausage.csv
2. curr_dlping.csv
3. curr_dns.csv
4. curr_httpget.csv
5. curr_httpgetmt.csv
6. curr_httpgetmt6.csv
7. curr_httppost.csv
8. curr_httppostmt.csv
9. curr_httppostmt6.csv
10. curr_lct_dl.csv
11. curr_lct_ul.csv
12. curr_ping.csv
13. curr_traceroute.csv
14. curr_udpcloss.csv
15. curr_udpjitter.csv
16. curr_udplatency.csv
17. curr_udplatency6.csv
18. curr_ulping.csv
19. curr_webget.csv


In [8]:
import pandas as pd

print("🧹 Checking for empty files (0 rows):\n")

empty_files = []

for file in files_2023_deep:
    file_path = os.path.join(subfolder_2023, file)
    try:
        df = pd.read_csv(file_path, low_memory=False, nrows=1)
        if df.empty:
            empty_files.append(file)
    except Exception as e:
        print(f"{file}: ❌ Failed to read ({e})")

if empty_files:
    print("\n🚫 Empty files:")
    for f in empty_files:
        print(f"- {f}")
else:
    print("✅ No empty files found.")

🧹 Checking for empty files (0 rows):


🚫 Empty files:
- curr_httpget.csv
- curr_httpgetmt6.csv
- curr_httppost.csv


In [9]:
import pandas as pd

print("📊 Row count per file (data-raw-2023-jan/202301):\n")

for file in files_2023_deep:
    file_path = os.path.join(subfolder_2023, file)
    try:
        df = pd.read_csv(file_path, low_memory=False)
        print(f"{file}: {len(df):,} rows")
    except Exception as e:
        print(f"{file}: ❌ Failed to read ({e})")

📊 Row count per file (data-raw-2023-jan/202301):

curr_datausage.csv: 2,776,853 rows
curr_dlping.csv: 1,380,351 rows
curr_dns.csv: ❌ Failed to read (Error tokenizing data. C error: out of memory)
curr_httpget.csv: 0 rows
curr_httpgetmt.csv: 806,142 rows
curr_httpgetmt6.csv: 0 rows
curr_httppost.csv: 0 rows
curr_httppostmt.csv: 803,436 rows
curr_httppostmt6.csv: 2 rows
curr_lct_dl.csv: 977,725 rows
curr_lct_ul.csv: 975,854 rows
curr_ping.csv: 4,482,225 rows
curr_traceroute.csv: 13,265,007 rows
curr_udpcloss.csv: 2,138,359 rows
curr_udpjitter.csv: 2,688,577 rows
curr_udplatency.csv: 5,454,686 rows
curr_udplatency6.csv: 18,543 rows
curr_ulping.csv: 1,394,990 rows
curr_webget.csv: 10,668,402 rows


## 📘 Step 2: Data Understanding

In this step, we explore the structure, format, and contents of the key datasets. Our focus is to understand:

- Download/upload performance over time
- Relationships between internet quality metrics (latency, jitter, packet loss, etc.)
- Which features might serve as good predictors

### 📂 Datasets Selected for Analysis
Based on row counts and relevance to upload/download performance, we’ll focus on:
- `curr_datausage.csv` – likely includes bandwidth or data consumption
- `curr_lct_dl.csv` – likely download throughput
- `curr_lct_ul.csv` – likely upload throughput
- `curr_dlping.csv` / `curr_ulping.csv` – ping times (latency)
- `curr_udplatency.csv`, `curr_udpjitter.csv`, `curr_udpcloss.csv` – latency-related

These files contain millions of records, making them suitable for performance trend and modeling analysis.

Let’s preview a few of these datasets and inspect their structure using `.head()`, `.info()`, and `.describe()` methods.


In [10]:
crucial_files = ['curr_datausage.csv', 'curr_lct_dl.csv', 'curr_lct_ul.csv',
                 'curr_dlping.csv', 'curr_ulping.csv',
                 'curr_udplatency.csv', 'curr_udpjitter.csv', 'curr_udpcloss.csv']

for file in crucial_files:
    file_path = os.path.join(subfolder_2023, file)
    print(f"\nPreviewing {file}\n{'=' * 40}")

    try:
        df = pd.read_csv(file_path, low_memory=False)

        print("🔹 Head:")
        display(df.head())

        print("🔹 Info:")
        display(df.info())

        print("🔹 Describe:")
        display(df.describe())

    except Exception as e:
        print(f" Could not read {file}: {e}")


Previewing curr_datausage.csv
🔹 Head:


,unit_id,dtime,sk_tx_bytes,sk_rx_bytes,cust_wired_tx_bytes,cust_wired_rx_bytes,cust_wifi_tx_bytes,cust_wifi_rx_bytes
0,386,2023-01-01 06:32:00,0,61288,42114358,2190548237,0,3225217
1,386,2023-01-01 07:32:02,0,77942,36364678,1781312604,0,2878273
2,386,2023-01-01 08:32:03,1593,59292,31497966,914522306,0,2649831
3,386,2023-01-01 09:32:05,0,61601,37000182,1609290026,0,2889108
4,386,2023-01-01 10:32:07,2552,103223,33011116,1499945826,0,2771261


🔹 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2776853 entries, 0 to 2776852
Data columns (total 8 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   unit_id              int64 
 1   dtime                object
 2   sk_tx_bytes          int64 
 3   sk_rx_bytes          int64 
 4   cust_wired_tx_bytes  int64 
 5   cust_wired_rx_bytes  int64 
 6   cust_wifi_tx_bytes   int64 
 7   cust_wifi_rx_bytes   int64 
dtypes: int64(7), object(1)
memory usage: 169.5+ MB


None

🔹 Describe:


...

,unit_id,dtime,duration,target,address,packets
0,386,2023-01-21 06:05:08,4500449,newyorkfcc.west.verizon.net,206.124.86.197,2
1,386,2023-01-25 20:21:18,87004198,sp1-vm-newyork-us.samknows.com,151.139.31.1,57
2,390,2023-01-10 00:16:44,5277980,sp1-vm-newyork-us.samknows.com,151.139.31.1,3
3,390,2023-01-25 20:21:19,86999555,sp1-vm-newyork-us.samknows.com,151.139.31.1,57
4,390,2023-01-28 10:13:27,114962514,sp1-vm-newyork-us.samknows.com,151.139.31.1,76


🔹 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138359 entries, 0 to 2138358
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   unit_id   int64 
 1   dtime     object
 2   duration  int64 
 3   target    object
 4   address   object
 5   packets   int64 
dtypes: int64(3), object(3)
memory usage: 97.9+ MB


None

🔹 Describe:


,unit_id,duration,packets
count,2.138359e+06,2.138359e+06,2.138359e+06
mean,2.428527e+07,5.049842e+07,1.349578e+02
std,2.350328e+07,3.204128e+08,1.514010e+03
min,3.860000e+02,5.000000e+00,-2.000000e+00
25%,9.391880e+05,4.500053e+06,2.000000e+00
50%,2.575527e+07,4.744629e+06,2.000000e+00
75%,3.987683e+07,8.999804e+06,4.000000e+00
max,9.002113e+07,5.184420e+09,8.252330e+05
